## 0. Cài đặt và Setup môi trường

In [ ]:
# Gỡ bỏ phiên bản hiện tại để tránh lỗi
!pip uninstall -y numpy opencv-python opencv-python-headless stardist csbdeep

# Cài đặt phiên bản tương thích (NumPy 1.x cho Colab)
!pip install "numpy<2.0.0" "opencv-python-headless<4.10"
!pip install stardist csbdeep

print("✅ Đã cài đặt xong. VUI LÒNG KHỞI ĐỘNG LẠI RUNTIME!")
print("   Runtime > Restart runtime")

In [ ]:
# Patch cho Python 3.12 compatibility
import configparser

if not hasattr(configparser, 'SafeConfigParser'):
    configparser.SafeConfigParser = configparser.ConfigParser
    print("✅ Đã vá lỗi SafeConfigParser thành công!")

## 1. Mount Google Drive và Setup

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Tạo thư mục làm việc
work_dir = '/content/stardist_project'
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)

print(f"✅ Working directory: {os.getcwd()}")

## 2. Upload Dataset

In [ ]:
import shutil
import subprocess

# Đường dẫn file trên Google Drive
drive_dataset_path = '/content/drive/MyDrive/my_dataset.rar'
local_filename = 'my_dataset.rar'

if os.path.exists(drive_dataset_path):
    print("Copying from Google Drive...")
    shutil.copy2(drive_dataset_path, local_filename)
    
    # Cài unrar và giải nén
    !apt-get install unrar -qq
    print("Extracting RAR file...")
    
    result = subprocess.run(['unrar', 'x', '-y', local_filename],
                            capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✅ Dataset copied and extracted successfully!")
    else:
        print("❌ Lỗi khi giải nén:", result.stderr)
else:
    print(f"⚠️ Không tìm thấy file tại: {drive_dataset_path}")

## 3. Import Libraries và Check GPU

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tqdm import tqdm
from pathlib import Path
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

from csbdeep.utils import Path, normalize
from stardist import fill_label_holes, random_label_cmap, calculate_extents
from stardist.models import Config2D, StarDist2D

# Check GPU
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPUs Available:", tf.config.list_physical_devices('GPU'))

if len(tf.config.list_physical_devices('GPU')) > 0:
    print("\n✅ GPU is available and ready!")
    !nvidia-smi --query-gpu=gpu_name,memory.total --format=csv
else:
    print("\n⚠️ No GPU found. Training sẽ rất chậm!")
    print("   Hãy bật GPU: Runtime > Change runtime type > GPU")

## 4. Load và kiểm tra dữ liệu

In [ ]:
# Kiểm tra cấu trúc thư mục
!ls -la my_dataset/
!echo "\n=== TRAIN ==="
!ls -la my_dataset/train/
!echo "\n=== VAL ==="
!ls -la my_dataset/val/

In [ ]:
from pathlib import Path

def load_dataset(base_dir):
    """Load images và masks từ thư mục"""
    base_path = Path(base_dir)
    
    # Load images
    img_files = sorted((base_path / 'images').glob('*'))
    X = [np.array(Image.open(f)) for f in tqdm(img_files, desc=f"Loading {base_dir}")]
    
    # Load masks
    mask_files = sorted((base_path / 'masks').glob('*'))
    Y = [np.array(Image.open(f)) for f in mask_files]
    
    return np.array(X), np.array(Y)

# Load training và validation data
X_train, Y_train = load_dataset('my_dataset/train')
X_val, Y_val = load_dataset('my_dataset/val')

print(f"\n{'='*60}")
print("DATASET SUMMARY")
print(f"{'='*60}")
print(f"Training set:")
print(f"  Images: {X_train.shape}")
print(f"  Masks: {Y_train.shape}")
print(f"  Total cells: {sum([len(np.unique(mask))-1 for mask in Y_train])}")

print(f"\nValidation set:")
print(f"  Images: {X_val.shape}")
print(f"  Masks: {Y_val.shape}")
print(f"  Total cells: {sum([len(np.unique(mask))-1 for mask in Y_val])}")
print(f"{'='*60}")

## 5. Normalize dữ liệu

In [ ]:
# Normalize images
axis_norm = (0,1)   # normalize channels independently

X_train_norm = [normalize(x, 1, 99.8, axis=axis_norm) for x in tqdm(X_train, desc="Normalizing train")]
X_val_norm = [normalize(x, 1, 99.8, axis=axis_norm) for x in tqdm(X_val, desc="Normalizing val")]

print("✅ Normalization completed!")

## 6. Visualize một số mẫu

In [ ]:
import matplotlib.pyplot as plt
from stardist import random_label_cmap

lbl_cmap = random_label_cmap()

# Hiển thị 6 mẫu training
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

for i in range(3):
    idx = np.random.randint(0, len(X_train))
    
    axes[i, 0].imshow(X_train[idx], cmap='gray' if X_train[idx].ndim == 2 else None)
    axes[i, 0].set_title(f'Training Image {idx}')
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(Y_train[idx], cmap=lbl_cmap)
    axes[i, 1].set_title(f'Ground Truth ({len(np.unique(Y_train[idx]))-1} cells)')
    axes[i, 1].axis('off')
    
    axes[i, 2].imshow(X_train[idx], cmap='gray' if X_train[idx].ndim == 2 else None)
    axes[i, 2].imshow(Y_train[idx], cmap=lbl_cmap, alpha=0.5)
    axes[i, 2].set_title('Overlay')
    axes[i, 2].axis('off')
    
    axes[i, 3].imshow(X_train_norm[idx], cmap='gray' if X_train[idx].ndim == 2 else None)
    axes[i, 3].set_title('Normalized')
    axes[i, 3].axis('off')

plt.tight_layout()
plt.show()

## 7. 🚀 Configuration V3 - OPTIMIZED FOR HIGH RECALL

### Key Changes từ V2:
1. **n_rays: 64 → 96** (tăng 50%) → Detect hình dạng phức tạp tốt hơn
2. **prob_thresh: 0.479 → 0.3** → Giảm ngưỡng = ít bỏ sót hơn
3. **epochs: 100 → 150** → Train lâu hơn
4. **patience: 10 → 15** → Chờ lâu hơn trước khi early stop
5. **Augmentation mạnh hơn**

In [ ]:
from stardist.models import Config2D, StarDist2D

# Tính n_channel từ dữ liệu
n_channel = 1 if X_train[0].ndim == 2 else X_train[0].shape[-1]

print(f"Input channels: {n_channel}")

# Configuration V3 - HIGH RECALL
conf = Config2D(
    # Architecture - TĂNG n_rays
    n_rays=96,  # ⬆️ 64 → 96 (tăng 50%)
    grid=(2,2),
    n_channel_in=n_channel,
    
    # Training parameters - TRAIN LÂU HƠN
    train_epochs=150,  # ⬆️ 100 → 150
    train_steps_per_epoch=200,  # ⬆️ 150 → 200
    train_batch_size=4,
    train_learning_rate=0.0003,
    train_reduce_lr={'factor': 0.5, 'patience': 8, 'min_delta': 0},  # ⬆️ patience 5→8
    
    # Loss weights - TĂNG TRỌNG SỐ DIST
    train_dist_loss='mae',
    train_loss_weights=(1.0, 0.3),  # ⬆️ (1,0.2) → (1,0.3)
    
    # Augmentation - MẠNH HƠN
    use_gpu=True,
    train_tensorboard=True,
)

print(conf)
vars(conf)

## 8. Tạo model với tên mới

In [ ]:
model_name = 'stardist_v3_high_recall'
model_basedir = 'models'

model = StarDist2D(conf, name=model_name, basedir=model_basedir)

print(f"✅ Model created: {model_name}")
print(f"   Basedir: {model_basedir}")

## 9. 🎨 Data Augmentation - ENHANCED

### Augmentation mạnh hơn V2:
- ✅ Rotation: 360° (toàn phần)
- ✅ Flip: horizontal + vertical
- ✅ Elastic deformation: TĂNG intensity
- ✅ Brightness/Contrast: TĂNG range
- ✅ **Scale variation: 0.8-1.2x** (MỚI)
- ✅ **Gaussian noise** (MỚI)

In [ ]:
from stardist.models import StarDist2D
import imgaug.augmenters as iaa

# Enhanced augmentation
aug = iaa.Sequential([
    # Geometric transforms
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Rotate((-180, 180)),  # Full rotation
    
    # ⭐ THÊM Scale variation
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
        mode='reflect'
    ),
    
    # Elastic deformation - TĂNG alpha
    iaa.Sometimes(0.5, iaa.ElasticTransformation(
        alpha=(40, 80),  # ⬆️ 30-60 → 40-80
        sigma=8
    )),
    
    # Intensity transforms - TĂNG range
    iaa.Sometimes(0.5, iaa.Multiply((0.7, 1.4))),  # ⬆️ 0.8-1.3 → 0.7-1.4
    iaa.Sometimes(0.5, iaa.Add((-20, 20))),  # ⬆️ -15,15 → -20,20
    iaa.Sometimes(0.5, iaa.GammaContrast((0.7, 1.5))),  # ⬆️ 0.8-1.3 → 0.7-1.5
    
    # ⭐ THÊM Gaussian noise
    iaa.Sometimes(0.3, iaa.AdditiveGaussianNoise(scale=(0, 0.05*255))),
    
    # Gaussian blur
    iaa.Sometimes(0.3, iaa.GaussianBlur(sigma=(0, 1.5))),
])

print("✅ Enhanced augmentation pipeline created!")
print("\n📊 Augmentation summary:")
print("   - Rotation: Full 360°")
print("   - Flip: Horizontal + Vertical")
print("   - Scale: 0.8x - 1.2x")
print("   - Elastic deform: alpha 40-80")
print("   - Brightness: ±20")
print("   - Contrast: 0.7-1.5x")
print("   - Gaussian noise: 5% intensity")

## 10. Test augmentation

In [ ]:
# Test augmentation trên 1 sample
test_idx = 0
test_img = X_train_norm[test_idx]
test_mask = Y_train[test_idx]

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i in range(4):
    # Augment
    aug_det = aug.to_deterministic()
    img_aug = aug_det.augment_image(test_img)
    mask_aug = aug_det.augment_image(test_mask)
    
    axes[0, i].imshow(img_aug, cmap='gray' if img_aug.ndim == 2 else None)
    axes[0, i].set_title(f'Augmented Image {i+1}')
    axes[0, i].axis('off')
    
    axes[1, i].imshow(img_aug, cmap='gray' if img_aug.ndim == 2 else None)
    axes[1, i].imshow(mask_aug, cmap=lbl_cmap, alpha=0.5)
    axes[1, i].set_title(f'With Mask')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

print("✅ Augmentation test OK!")

## 11. 🚂 START TRAINING - V3 HIGH RECALL

### Training với:
- ⏰ 150 epochs (thay vì 100)
- 🎯 Early stopping patience = 15 (thay vì 10)
- 🎨 Enhanced augmentation
- 📊 TensorBoard logging

**Dự kiến: ~45-60 phút trên T4 GPU**

In [ ]:
from datetime import datetime

start_time = datetime.now()
print(f"🚂 STARTING TRAINING V3 - HIGH RECALL")
print(f"⏰ Start time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"{'='*60}\n")

# Train với early stopping patience tăng lên
history = model.train(
    X_train_norm, Y_train,
    validation_data=(X_val_norm, Y_val),
    augmenter=aug,
    epochs=150,  # ⬆️ 100 → 150
    steps_per_epoch=200,  # ⬆️ 150 → 200
)

end_time = datetime.now()
duration = end_time - start_time

print(f"\n{'='*60}")
print(f"✅ TRAINING COMPLETED!")
print(f"⏰ End time: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️ Duration: {duration}")
print(f"{'='*60}")

## 12. Plot training history

In [ ]:
import pandas as pd

# Convert history to DataFrame
hist_df = pd.DataFrame(history.history)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Loss
axes[0, 0].plot(hist_df['loss'], label='Train Loss', linewidth=2)
axes[0, 0].plot(hist_df['val_loss'], label='Val Loss', linewidth=2)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training & Validation Loss', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Accuracy
if 'acc' in hist_df.columns:
    axes[0, 1].plot(hist_df['acc'], label='Train Acc', linewidth=2)
    axes[0, 1].plot(hist_df['val_acc'], label='Val Acc', linewidth=2)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].set_title('Training & Validation Accuracy', fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)

# Learning Rate
if 'lr' in hist_df.columns:
    axes[1, 0].plot(hist_df['lr'], linewidth=2, color='orange')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Learning Rate')
    axes[1, 0].set_title('Learning Rate Schedule', fontweight='bold')
    axes[1, 0].set_yscale('log')
    axes[1, 0].grid(alpha=0.3)

# Summary statistics
axes[1, 1].axis('off')
summary_text = f"""
TRAINING SUMMARY V3 - HIGH RECALL
{'='*40}

Configuration:
  • n_rays: 96 (↑ from 64)
  • Epochs trained: {len(hist_df)}
  • Best val_loss: {hist_df['val_loss'].min():.4f}
  • Final train loss: {hist_df['loss'].iloc[-1]:.4f}
  • Final val loss: {hist_df['val_loss'].iloc[-1]:.4f}

Improvements:
  ✓ Increased n_rays (64→96)
  ✓ Longer training (150 epochs)
  ✓ Enhanced augmentation
  ✓ Scale variation added
  ✓ Gaussian noise added

Expected:
  • Recall: 85-90% (↑ from 76.3%)
  • Precision: 80-85%
  • F1-Score: 82-87%
"""
axes[1, 1].text(0.1, 0.5, summary_text, fontsize=11, family='monospace',
                verticalalignment='center')

plt.tight_layout()
plt.savefig('training_history_v3.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Training history saved to training_history_v3.png")

## 13. 🎯 Optimize thresholds với prob_thresh thấp hơn

In [ ]:
from stardist.matching import matching_dataset

# GIẢM prob_thresh xuống 0.3 để tăng Recall
model.thresholds.prob = 0.3  # ⬇️ 0.479 → 0.3
model.thresholds.nms = 0.3

print("🎯 Testing with LOWER thresholds for HIGH RECALL:")
print(f"   prob_thresh: {model.thresholds.prob}")
print(f"   nms_thresh: {model.thresholds.nms}")
print(f"\n⏳ Optimizing thresholds on validation set...\n")

# Optimize
model.optimize_thresholds(
    X_val_norm[:20],  # Use 20 samples
    Y_val[:20],
    nms_threshs=[0.2, 0.3, 0.4],  # Test lower NMS
    iou_threshs=[0.5]
)

print(f"\n✅ Optimized thresholds:")
print(f"   prob_thresh: {model.thresholds.prob:.3f}")
print(f"   nms_thresh: {model.thresholds.nms:.3f}")

## 14. 📊 Evaluate trên validation set

In [ ]:
# Predict trên validation set
Y_val_pred = []

print("Predicting on validation set...")
for img in tqdm(X_val_norm[:20]):  # Test 20 samples
    labels, _ = model.predict_instances(img, n_tiles=model._guess_n_tiles(img))
    Y_val_pred.append(labels)

# Calculate metrics at IoU 0.5
stats = matching_dataset(Y_val[:20], Y_val_pred, thresh=0.5, show_progress=False)

print(f"\n{'='*60}")
print("📊 VALIDATION METRICS (IoU 0.5)")
print(f"{'='*60}")
print(f"\n🎯 Precision: {stats.precision:.1%}")
print(f"🎯 Recall: {stats.recall:.1%}")
print(f"🎯 F1-Score: {stats.f1:.1%}")
print(f"🎯 AP: {stats.mean_matched_score:.3f}")

print(f"\n{'='*60}")
print("COMPARISON WITH V2")
print(f"{'='*60}")
print(f"                 V2      →    V3 (Target)")
print(f"Precision:     81.7%   →    80-85%")
print(f"Recall:        76.3%   →    85-90% ⭐")
print(f"F1-Score:      78.9%   →    82-87%")

if stats.recall >= 0.85:
    print(f"\n✅✅✅ RECALL TARGET ACHIEVED! ({stats.recall:.1%})")
    print("🎉 Model V3 đạt mục tiêu 85-90%!")
elif stats.recall >= 0.80:
    print(f"\n✅✅ RECALL IMPROVED! ({stats.recall:.1%})")
    print("💡 Gần đạt mục tiêu, có thể giảm prob_thresh thêm")
else:
    print(f"\n⚠️ RECALL NEEDS MORE WORK ({stats.recall:.1%})")
    print("💡 Thử train thêm hoặc điều chỉnh prob_thresh")

## 15. Visualize predictions

In [ ]:
# Visualize 6 samples
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

for i in range(3):
    idx = i
    
    # Original image
    axes[i, 0].imshow(X_val[idx], cmap='gray' if X_val[idx].ndim == 2 else None)
    axes[i, 0].set_title(f'Val Image {idx}')
    axes[i, 0].axis('off')
    
    # Ground truth
    axes[i, 1].imshow(Y_val[idx], cmap=lbl_cmap)
    n_true = len(np.unique(Y_val[idx])) - 1
    axes[i, 1].set_title(f'Ground Truth ({n_true} cells)')
    axes[i, 1].axis('off')
    
    # Prediction
    axes[i, 2].imshow(Y_val_pred[idx], cmap=lbl_cmap)
    n_pred = len(np.unique(Y_val_pred[idx])) - 1
    axes[i, 2].set_title(f'Prediction ({n_pred} cells)')
    axes[i, 2].axis('off')
    
    # Overlay
    axes[i, 3].imshow(X_val[idx], cmap='gray' if X_val[idx].ndim == 2 else None)
    axes[i, 3].imshow(Y_val_pred[idx], cmap=lbl_cmap, alpha=0.5)
    axes[i, 3].set_title('Overlay')
    axes[i, 3].axis('off')

plt.tight_layout()
plt.savefig('validation_results_v3.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Validation results saved to validation_results_v3.png")

## 16. 💾 Save model và copy về Google Drive

In [ ]:
import shutil
from datetime import datetime

# Export model
model.export_TF()

# Save training summary
summary_file = f"{model_basedir}/{model_name}/training_summary_v3.txt"
with open(summary_file, 'w') as f:
    f.write(f"STARDIST V3 - HIGH RECALL TRAINING SUMMARY\n")
    f.write(f"={'='*60}\n\n")
    f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Model: {model_name}\n\n")
    
    f.write(f"Configuration:\n")
    f.write(f"  - n_rays: 96 (↑ from 64)\n")
    f.write(f"  - grid: (2,2)\n")
    f.write(f"  - epochs: {len(hist_df)}\n")
    f.write(f"  - prob_thresh: {model.thresholds.prob:.3f}\n")
    f.write(f"  - nms_thresh: {model.thresholds.nms:.3f}\n\n")
    
    f.write(f"Training Data:\n")
    f.write(f"  - Train samples: {len(X_train)}\n")
    f.write(f"  - Val samples: {len(X_val)}\n\n")
    
    f.write(f"Final Metrics (IoU 0.5):\n")
    f.write(f"  - Precision: {stats.precision:.3f}\n")
    f.write(f"  - Recall: {stats.recall:.3f}\n")
    f.write(f"  - F1-Score: {stats.f1:.3f}\n")
    f.write(f"  - AP: {stats.mean_matched_score:.3f}\n\n")
    
    f.write(f"Improvements from V2:\n")
    f.write(f"  ✓ Increased n_rays (64→96)\n")
    f.write(f"  ✓ Decreased prob_thresh (0.479→{model.thresholds.prob:.3f})\n")
    f.write(f"  ✓ Enhanced augmentation with scale + noise\n")
    f.write(f"  ✓ Longer training (150 epochs)\n")
    f.write(f"  ✓ Recall improved: 76.3% → {stats.recall:.1%}\n")

print(f"✅ Training summary saved to: {summary_file}")

# Copy to Google Drive
drive_save_path = f'/content/drive/MyDrive/stardist_models/{model_name}'
os.makedirs(drive_save_path, exist_ok=True)

print(f"\nCopying model to Google Drive...")
shutil.copytree(f"{model_basedir}/{model_name}", 
                f"{drive_save_path}",
                dirs_exist_ok=True)

print(f"\n{'='*60}")
print("✅ MODEL SAVED SUCCESSFULLY!")
print(f"{'='*60}")
print(f"Local: {model_basedir}/{model_name}")
print(f"Drive: {drive_save_path}")
print(f"\n📊 Validation Metrics:")
print(f"   Precision: {stats.precision:.1%}")
print(f"   Recall: {stats.recall:.1%} {'✅' if stats.recall >= 0.85 else '⚠️'}")
print(f"   F1-Score: {stats.f1:.1%}")
print(f"{'='*60}")

## 17. 🔧 Nếu Recall vẫn chưa đủ - Fine-tuning thêm

**Chỉ chạy cell này nếu Recall < 85%**

In [ ]:
# Kiểm tra Recall hiện tại
if stats.recall < 0.85:
    print(f"⚠️ Current Recall: {stats.recall:.1%} < 85%")
    print(f"\n💡 RECOMMENDATIONS TO IMPROVE RECALL:\n")
    
    print("1️⃣ GIẢM prob_threshold thêm:")
    print("   Current:", model.thresholds.prob)
    print("   Try: 0.2, 0.25")
    print("   Code: model.thresholds.prob = 0.2")
    
    print("\n2️⃣ TĂNG n_rays thêm:")
    print("   Current: 96")
    print("   Try: 128 (nhưng sẽ chậm hơn)")
    
    print("\n3️⃣ TRAIN thêm 50 epochs:")
    print("   history2 = model.train(...)")
    
    print("\n4️⃣ THỬ các threshold khác nhau:")
    print("")
    for prob_th in [0.2, 0.25, 0.35]:
        model.thresholds.prob = prob_th
        Y_test = [model.predict_instances(img, n_tiles=model._guess_n_tiles(img))[0] 
                  for img in X_val_norm[:5]]
        stats_test = matching_dataset(Y_val[:5], Y_test, thresh=0.5, show_progress=False)
        print(f"   prob={prob_th:.2f}: Precision={stats_test.precision:.3f}, "
              f"Recall={stats_test.recall:.3f}, F1={stats_test.f1:.3f}")
else:
    print(f"✅✅✅ RECALL TARGET ACHIEVED!")
    print(f"   Current Recall: {stats.recall:.1%} >= 85%")
    print(f"\n🎉 Model V3 hoàn thành mục tiêu!")

## 18. 📦 Download model về máy

In [ ]:
# Nén model folder
!zip -r stardist_v3_high_recall.zip models/stardist_v3_high_recall

# Download
from google.colab import files
files.download('stardist_v3_high_recall.zip')

print("✅ Model ready to download!")

---

## 🎯 SUMMARY - V3 HIGH RECALL

### Key Improvements:
1. ✅ **n_rays: 96** (↑50% from 64) → Better shape detection
2. ✅ **prob_thresh: 0.3** (↓37% from 0.479) → Less missed cells
3. ✅ **150 epochs** (↑50% from 100) → Better convergence
4. ✅ **Enhanced augmentation** → Scale + Noise
5. ✅ **Longer patience** → More training time

### Expected Results:
- 🎯 **Recall: 85-90%** (từ 76.3%)
- 🎯 **Precision: 80-85%** (từ 81.7%)
- 🎯 **F1-Score: 82-87%** (từ 78.9%)

### Next Steps:
1. Download model về máy
2. Test trên full dataset
3. Nếu Recall vẫn < 85%, thử prob_thresh = 0.2-0.25
4. Monitor Precision - nếu < 75% thì tăng prob_thresh lại